In [1]:
# Code written by: Michael Bramble | michael.s.bramble@jpl.nasa.gov
# unmixing test script for EMIT AMD investigation
# copying process from my CRISM/VIR studies
# 20240124 - initial version

# general process
# (pull image info form a list and being a loop)
# load reflectance image and geometry file (phocube)
# load endmembers for unmixing (endmember script massages lab spectra to match instrument, also ssa?)
# subset data to relevant wavelenth range
# calculate mean-column endmember?
# begin nested loop through each line and sample
# pull the pixel's reflectance specturm (convert to ssa?)
# perform linear least squares unmixing, return fractions and rmse
# save maps of spectral fraction and rmse
# save visualizations
# (end one image and loop iteration)

In [19]:
# import earthaccess
import os
# from osgeo import gdal
import numpy as np
import pandas as pd
import xarray as xr
from scipy.optimize import lsq_linear
# import math
# import rasterio as rio
# import holoviews as hv
import hvplot.xarray
# import netCDF4 as nc
import sys
sys.path.append('/Users/bramble/My Drive/_JPL_AMD/EMIT-Data-Resources-main/python/modules/')
from emit_tools import emit_xarray

In [2]:
# load reflectance image
granule_asset_id = 'EMIT_L2A_RFL_001_20231006T174849_2327912_017.nc'
fp = f'/Users/bramble/Documents/emit/leadville/ref/{granule_asset_id}'
# load data set and remove the bad bands
ds_geo = emit_xarray(fp, ortho=True)
ds_geo['reflectance'].data[:,:,ds_geo['good_wavelengths'].data==0] = np.nan

# as a test, plot spectra taken from the georeferenced image
point = ds_geo.sel(longitude=-106.5,latitude=39,method='nearest')
ds_geo.sel(wavelengths=850, method='nearest').hvplot.image(cmap='viridis', aspect = 'equal', rasterize=True) +\
point.hvplot.line(y='reflectance',x='wavelengths', color='black', frame_width=400).opts(
    title = f'Latitude = {point.latitude.values.round(3)}, Longitude = {point.longitude.values.round(3)}')

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


BokehModel(combine_events=True, render_bundle={'docs_json': {'9ec349d3-bf40-476a-91f2-da6a238d0d8c': {'version…

Task exception was never retrieved
future: <Task finished name='Task-7' coro=<Callback.process_on_change() done, defined at /Users/bramble/opt/miniconda3/envs/emit/lib/python3.9/site-packages/holoviews/plotting/bokeh/callbacks.py:322> exception=UnsetValueError("figure(id='p1010', ...).inner_height doesn't have a value set")>
Traceback (most recent call last):
  File "/Users/bramble/opt/miniconda3/envs/emit/lib/python3.9/site-packages/holoviews/plotting/bokeh/callbacks.py", line 340, in process_on_change
    msg[attr] = self.resolve_attr_spec(path, cb_obj)
  File "/Users/bramble/opt/miniconda3/envs/emit/lib/python3.9/site-packages/holoviews/plotting/bokeh/callbacks.py", line 248, in resolve_attr_spec
    resolved = getattr(resolved, p, None)
  File "/Users/bramble/opt/miniconda3/envs/emit/lib/python3.9/site-packages/bokeh/core/property/descriptors.py", line 283, in __get__
    raise UnsetValueError(f"{obj}.{self.name} doesn't have a value set")
bokeh.core.property.descriptors.UnsetValue

In [3]:
# SPATIAL SUBSET IMAGE HERE 
# FOR NOW, MANUALLY. IN FUTURE, AUTOMATE

# LEADVILLE
max_lat = 39.3
min_lat = 39.2
max_lon = -106.3
min_lon = -106.2

# simplify the ref band
band = ds_geo.reflectance

# select just the spatial subset around the mine
subset_data = band.sel({'latitude' : slice(max_lat, min_lat),
                      'longitude' : slice(max_lon, min_lon)})

# GET LAT/LON ARRAYS
band_lon =  subset_data['longitude']
band_lat =  subset_data['latitude']

# REPALCE NaNs. Likely not needed in the final version.
subset_data = subset_data.fillna(0)

# # DISPLAY SUBSET DATA
# subset_data.sel(wavelengths=850, method='nearest').hvplot.image(cmap='viridis', frame_width=500, geo=True, tiles='EsriImagery').opts(
#     xlabel=f'{ds_geo.longitude.long_name} ({ds_geo.longitude.units})', ylabel=f'{ds_geo.latitude.long_name} ({ds_geo.latitude.units})')

# PLOT SPECTRUM FROM SUBSET DATA USING INDEXES
subset_data[50,50].hvplot.line(y='reflectance',x='wavelengths', color='black')

:Curve   [wavelengths]   (reflectance)

In [21]:
# load wavelenth array
file = '/Users/bramble/My Drive/_JPL_AMD/endmembers/emit_wavelengths.txt'
wavelengths_emit_nm = pd.read_csv(file,header=None)
wavelengths_emit_um = wavelengths_emit_nm[0]/1000

# load endmember library
from script_py_load_amd_endmembers import endmembers
endmember_array = np.arange(1,endmembers.shape[1]+1,1)
endmembers_xr = xr.DataArray(endmembers, coords=[("wavelengths", wavelengths_emit_um), ("endmembers", endmember_array)])

<xarray.DataArray (wavelengths: 285, endmembers: 8)>
array([[0.0363389 , 0.0363389 , 0.01873823, ..., 0.00886171, 0.03487126,
        0.01472756],
       [0.03979002, 0.03979002, 0.01923035, ..., 0.00894847, 0.04039592,
        0.01453052],
       [0.04466611, 0.04466611, 0.01956765, ..., 0.00933741, 0.05129526,
        0.01458615],
       ...,
       [0.06957571, 0.06957571, 0.74001258, ..., 0.31010951, 0.06032468,
        0.31407272],
       [0.06919427, 0.06919427, 0.73953161, ..., 0.30670942, 0.05909484,
        0.31089289],
       [0.06878318, 0.06878318, 0.73870336, ..., 0.30355727, 0.05781409,
        0.30782539]])
Coordinates:
  * wavelengths  (wavelengths) float64 0.381 0.3884 0.3958 ... 2.478 2.486 2.493
  * endmembers   (endmembers) int64 1 2 3 4 5 6 7 8

In [23]:
# perform spectral subset on EMIT image and endmember array


test = subset_data.sel(wavelengths=slice(500, 800))
test2 = endmembers_xr.sel(wavelengths=slice(0.5, 0.8))





<xarray.DataArray (wavelengths: 40, endmembers: 8)>
array([[0.09173164, 0.09173164, 0.02104398, 0.04812163, 0.34627923,
        0.03294947, 0.28892511, 0.02030895],
       [0.09471617, 0.09471617, 0.02157718, 0.05700778, 0.36417262,
        0.03819311, 0.29691166, 0.02160241],
       [0.09633177, 0.09633177, 0.02231277, 0.06517727, 0.3839216 ,
        0.04319173, 0.30060325, 0.02381948],
       [0.09814731, 0.09814731, 0.02256359, 0.07668101, 0.40558679,
        0.04961735, 0.3038041 , 0.02430497],
       [0.10041743, 0.10041743, 0.02381894, 0.09140277, 0.42806127,
        0.05662628, 0.30936623, 0.02853171],
       [0.10254627, 0.10254627, 0.02519392, 0.10760858, 0.45136223,
        0.06533606, 0.31459091, 0.02881052],
       [0.10420818, 0.10420818, 0.02804074, 0.12477768, 0.47566112,
        0.07359872, 0.31995067, 0.03072432],
       [0.1056935 , 0.1056935 , 0.03388362, 0.14229682, 0.4991198 ,
        0.08268521, 0.32750898, 0.03366112],
       [0.10711082, 0.10711082, 0.04391384, 0.16019187, 0.5142144 ,
        0.09213828, 0.33609544, 0.03703347],
       [0.10852206, 0.10852206, 0.06117726, 0.17343148, 0.52779976,
        0.10355501, 0.34389148, 0.04011126],
...
       [0.11599357, 0.11599357, 0.30387386, 0.26535321, 0.72715958,
        0.29413443, 0.3951587 , 0.16445935],
       [0.11548233, 0.11548233, 0.30980535, 0.27021725, 0.71277539,
        0.29861402, 0.38611286, 0.16961677],
       [0.11488182, 0.11488182, 0.31158921, 0.27770279, 0.698906  ,
        0.30186431, 0.37699343, 0.17559532],
       [0.11436398, 0.11436398, 0.30995155, 0.28048041, 0.6790076 ,
        0.3057118 , 0.36968268, 0.18049492],
       [0.11359422, 0.11359422, 0.30536454, 0.28259761, 0.66054282,
        0.30676555, 0.35935421, 0.18386183],
       [0.11278462, 0.11278462, 0.2991935 , 0.28348206, 0.6422172 ,
        0.30751804, 0.35166393, 0.18650306],
       [0.11201681, 0.11201681, 0.29061056, 0.28297491, 0.62269951,
        0.30658645, 0.34307318, 0.18954408],
       [0.111056  , 0.111056  , 0.28288428, 0.2778698 , 0.60443691,
        0.30643343, 0.33339898, 0.1914252 ],
       [0.11039022, 0.11039022, 0.27384372, 0.27503633, 0.58679975,
        0.3055053 , 0.32550106, 0.19210396],
       [0.1093568 , 0.1093568 , 0.26544966, 0.26904542, 0.56876435,
        0.30294366, 0.31799552, 0.1925649 ]])
Coordinates:
  * wavelengths  (wavelengths) float64 0.5072 0.5147 0.5221 ... 0.7904 0.7979
  * endmembers   (endmembers) int64 1 2 3 4 5 6 7 8

In [7]:
# test = subset_data[50,50]
# test
# subset_data.coords['latitude']
# subset_data.coords['longitude']
# subset_data
# newarray = xr.DataArray(None, coords=dict(latitude=subset_data.coords['latitude'],longitude=subset_data.coords['longitude']),dims=[subset_data.dims[0],subset_data.dims[1]])
# newarray

In [100]:
# developing empty array for the model outputs

# endmember_array = np.arange(1,endmembers.shape[1]+1,1)
# subset_data.coords['latitude']

# endmembers_coords = [("endmembers", np.arange(1,endmembers.shape[1]+1,1))]
# endmember_array = xr.DataArray(np.arange(1,endmembers.shape[1]+1,1),endmembers_coords,dims=["endmembers"])

# newarray_res_x = xr.DataArray(None, coords=dict(latitude=subset_data.coords['latitude'],longitude=subset_data.coords['longitude'],endmembers=endmember_array.coords['endmembers']),dims=[subset_data.dims[0],subset_data.dims[1],'endmembers'])
# newarray_res_x


<xarray.DataArray (latitude: 185, longitude: 185)>
array([[None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       ...,
       [None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None]], dtype=object)
Coordinates:
  * latitude   (latitude) float64 39.3 39.3 39.3 39.3 ... 39.2 39.2 39.2 39.2
  * longitude  (longitude) float64 -106.3 -106.3 -106.3 ... -106.2 -106.2 -106.2

In [107]:
# begin unmixing process


# newarray = xr.DataArray(None, subset_data.coords, subset_data.dims)
newarray_res_x = xr.DataArray(None, coords=dict(latitude=subset_data.coords['latitude'],longitude=subset_data.coords['longitude'],endmembers=endmember_array.coords['endmembers']),dims=[subset_data.dims[0],subset_data.dims[1],'endmembers'])
newarray_res_opt = xr.DataArray(None, coords=dict(latitude=subset_data.coords['latitude'],longitude=subset_data.coords['longitude']),dims=[subset_data.dims[0],subset_data.dims[1]])
for i in np.arange(0,len(band_lon)-1):
    for j in np.arange(0,len(band_lat)-1):
        spectrum = subset_data[i,j]

        res = lsq_linear(endmembers, spectrum, bounds=(0, 1), lsmr_tol='auto', verbose=0)

        res.x[res.x < 0.0001] = 0

        newarray_res_x[i,j] = res.x

        newarray_res_opt[i,j] = res.optimality






In [108]:

print(newarray_res_x[20,20])

print(newarray_res_opt[20,20])


<xarray.DataArray (endmembers: 8)>
array([0.13836079622132716, 0.13836079614990163, 0.0, 0.0, 0.0,
       0.045015827844514904, 0.1950437889875802, 0.0], dtype=object)
Coordinates:
    latitude    float64 39.29
    longitude   float64 -106.3
  * endmembers  (endmembers) int64 1 2 3 4 5 6 7 8
<xarray.DataArray ()>
array(5.668653084780885e-12, dtype=object)
Coordinates:
    latitude   float64 39.29
    longitude  float64 -106.3


In [10]:
# import numpy as np
# from scipy.sparse import rand
# from scipy.optimize import lsq_linear
# rng = np.random.default_rng()

# m = 20000
# n = 10000

# A = rand(m, n, density=1e-4, random_state=rng)
# b = rng.standard_normal(m)

# lb = rng.standard_normal(n)
# ub = lb + 1

# res = lsq_linear(A, b, bounds=(lb, ub), lsmr_tol='auto', verbose=1)

The relative change of the cost function is less than `tol`.
Number of iterations 16, initial cost 1.4897e+04, final cost 1.0982e+04, first-order optimality 3.90e-08.


In [19]:
# res
# res.x
# res.fun
# res.optimality
# res.unbounded_sol

3.89634722357619e-08